In [ ]:
pip install google-cloud-bigquery

In [ ]:
pip install pandas

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
from google.oauth2.service_account import Credentials
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials


credentials = service_account.Credentials.from_service_account_file('creds')

project_id = 'level-2-test-task-newage'
client = bigquery.Client(credentials=credentials, project=project_id)

def query_bigquery(sql):
        query_job = client.query(sql)
        results = query_job.result()
        return [dict(row) for row in results]

def get_query(city):
    return f"""
    SELECT
      visitId,
      fullVisitorId,
      visitStartTime,
      date,
      totals.visits,
      geoNetwork.country,
      geoNetwork.region,
      geoNetwork.city,
      device.browser,
      device.operatingSystem,
      device.isMobile

    FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
    WHERE
      geoNetwork.city = '{city}'
    """

def fetch_data_concurrently(cities):
    results = {}
    with ThreadPoolExecutor(max_workers=len(cities)) as executor:
        futures = {executor.submit(query_bigquery, get_query(city)): city for city in cities}
        for future in as_completed(futures):
            city = futures[future]
            data = future.result()
            results[city] = data
    return results

cities = ["Sydney", "San Francisco", "London", "Tokyo","Chicago","Maharashtra", "Berlin"]

results = fetch_data_concurrently(cities)

dfs = []
for city, data in results.items():
    df = pd.DataFrame(data)
    df['city'] = city
    dfs.append(df)

united_df = pd.concat(dfs, ignore_index=True)

visitid_count = united_df.groupby(['country', 'region', 'city']).agg(
    visit_count=('visitId', 'count')
).reset_index()

unique_visitid_count = united_df.groupby(['country', 'region', 'city']).agg(
    unique_visitor_count=('fullVisitorId', pd.Series.nunique)
).reset_index()

user_count = pd.merge(visitid_count, unique_visitid_count, on=['country', 'region', 'city'])


scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('creds', scope)
client = gspread.authorize(credentials)

spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1CxuDtCA39TtHkc2KT-oiywkXjHZ_BRjNuB3_FJPNcko/edit#gid=0'
spreadsheet = client.open_by_url(spreadsheet_url)

united_df = pd.concat(dfs, ignore_index=True)

united_df['visitStartTime'] = pd.to_datetime(united_df['visitStartTime'], unit='s')

united_df = united_df.rename(columns={
    'visitId': 'Visit ID',
    'fullVisitorId': 'Full Visitor ID',
    'visitStartTime': 'Visit Start Time',
    'date': 'Date',
    'totals.visits': 'Total Visits',
    'geoNetwork.country': 'Country',
    'geoNetwork.region': 'Region',
    'geoNetwork.city': 'City',
    'device.browser': 'Browser',
    'device.operatingSystem': 'Operating System',
    'device.isMobile': 'Is Mobile'
})

visitid_count = united_df.groupby(['country', 'region', 'city']).agg(visit_count=('Visit ID', 'count')).reset_index()

unique_visitid_count = united_df.groupby(['country', 'region', 'city']).agg(unique_visitor_count=('Full Visitor ID', pd.Series.nunique)).reset_index()
user_count = pd.merge(visitid_count, unique_visitid_count, on=['country', 'region', 'city'])

user_count_by_country_and_device = united_df.groupby(['country', 'isMobile'])['Full Visitor ID'].nunique().unstack(fill_value=0)

user_count_by_country_and_device['Total'] = user_count_by_country_and_device.sum(axis=1)

user_count_by_country_and_device = user_count_by_country_and_device.rename(columns={False: 'Desktop', True: 'Mobile'})

user_count_by_country_and_device = user_count_by_country_and_device.reset_index()

def write_to_sheet(worksheet_title, dataframe):
    worksheet = spreadsheet.add_worksheet(title=worksheet_title, rows="1000", cols="20")
    set_with_dataframe(worksheet, dataframe)

with ThreadPoolExecutor(max_workers=2) as executor:
    futures = []
    futures.append(executor.submit(write_to_sheet, 'Aggregated Data', united_df))
    futures.append(executor.submit(write_to_sheet, 'User Count by Country and Device', user_count_by_country_and_device))
    futures.append(executor.submit(write_to_sheet, 'User/Unique user count', user_count))

    for future in futures:
        future.result()

spreadsheet.del_worksheet(spreadsheet.get_worksheet(0))

print("OK")


OK
